## 84 - Monopoly Odds
> In the game, <strong>Monopoly</strong>, the standard board is set up in the following way:
    <div class="center">
    <img src="https://projecteuler.net/resources/images/0084_monopoly_board.png?1678992052" alt="0084_monopoly_board.png">
    </div>
    <p>A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.</p>
    <p>In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.</p>
    <p>At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.</p>
    <ul><li>Community Chest (2/16 cards):
    <ol><li>Advance to GO</li>
    <li>Go to JAIL</li>
    </ol></li>
    <li>Chance (10/16 cards):
    <ol><li>Advance to GO</li>
    <li>Go to JAIL</li>
    <li>Go to C1</li>
    <li>Go to E3</li>
    <li>Go to H2</li>
    <li>Go to R1</li>
    <li>Go to next R (railway company)</li>
    <li>Go to next R</li>
    <li>Go to next U (utility company)</li>
    <li>Go back 3 squares.</li>
    </ol></li>
    </ul><p>The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.</p>
    <p>By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.</p>
    <p>Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.</p>
    <p>If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.</p>

I simply simulated the game for a big number of steps. As I didn't want to use any modules, I implemented my own random number generator.

In [1]:
seed = 12345

def pseudo_random():
    '''
    Returns a pseudo-random float between 0 and 1 using the LCG algorithm
    '''
    global seed
    a = 1103515245
    c = 12345
    m = 2**31
    seed = (a * seed + c) % m
    return seed / m


def randint(lower, upper):
    """
    Returns a pseudo-random integer between lower and upper (inclusive)
    """
    range_size = upper - lower + 1
    return lower + int(pseudo_random() * range_size)

def shuffle(lst):
    """
    Shuffle the list 'lst' in place using the Fisher-Yates algorithm.
    """
    for i in range(len(lst) - 1, 0, -1):
        j = randint(0, i)
        lst[i], lst[j] = lst[j], lst[i]

dice_sides = 4
ch_positions = [0,10,11,24,39,5] # positions we can be sent to coming from a CH square

def next_railway(pos):
    '''
    Returns the position of the next railway station
    '''
    if pos<=5 or pos>35:
        return 5
    if pos<=15:
        return 15
    if pos<=25:
        return 25
    return 35

def next_utility(pos):
    '''
    Returns the position of the next utility company
    '''
    if pos<=12 or pos>28:
        return 12
    return 28

def next(pos, doubles, cc, ch):
    '''
    Returns the next state of the game.
    pos is the current position (between 0 and 39), doubles is the current number of doubles, cc is the current pile of CC cards, and ch is the current pile of CH cards.
    The function returns (pos, doubles, cc, ch, replay) where pos, doubles, cc, ch is the new state, and replay is a bool indicating if the player can replay immediately.
    '''
    dice1 = randint(1, dice_sides)
    dice2 = randint(1, dice_sides)

    # check if there is a double
    if dice1 == dice2:
        doubles += 1
        replay = True
        if doubles == 3:
            return (10, 0, cc, ch, False)
    else:
        doubles = 0
        replay = False
    
    # new position
    pos = (pos + dice1 + dice2) % 40

    # go to JAIL
    if pos==30:
        return (10,doubles,cc,ch,replay)
    
    # CC square
    elif pos in [2, 17, 33]:
        card = cc[0]
        cc = cc[1:] + [card] # the card is placed at the end of the deck
        if card == 0: # advance to GO
            return(0,doubles,cc,ch,replay)
        elif card == 1: # go to JAIL
            return(10,doubles,cc,ch,replay)
        else:
            return (pos,doubles,cc,ch,replay)
    
    # CH square
    elif pos in [7, 22, 36]:
        card = ch[0]
        ch = ch[1:] + [card] # the card is placed at the end of the deck
        if card <= 5:
            return(ch_positions[card],doubles,cc,ch,replay)
        elif card == 6 or card == 7: # next railway
            return (next_railway(pos),doubles,cc,ch,replay)
        elif card == 8: # next utility
            return (next_utility(pos),doubles,cc,ch,replay)
        elif card == 9: # go back 3 squares
            if pos == 36: # we land on a CC square
                card = cc[0]
                cc = cc[1:] + [card]
                if card == 0:
                    return(0,doubles,cc,ch,replay)
                elif card == 1:
                    return(10,doubles,cc,ch,replay)
                else:
                    return (33,doubles,cc,ch,replay)
            else:
                return ((pos-3)%40,doubles,cc,ch,replay)
        else:
            return (pos,doubles,cc,ch,replay)

    else:
        return (pos,doubles,cc,ch,replay)

def simulate(N):
    '''
    Simulates the game for N steps and returns the number of times we landed on each square
    '''
    probabilities = [0 for _ in range(40)]
    pos ,doubles = 0, 0
    cc = [i for i in range(16)]
    shuffle(cc)
    ch = [i for i in range(16)]
    shuffle(ch)
    for i in range(N):
        (pos,doubles,cc,ch,replay)=next(pos,doubles,cc,ch)
        while replay:
            (pos,doubles,cc,ch,replay)=next(pos,doubles,cc,ch)
        probabilities[pos]+=1
    return probabilities

def argmax(l):
    '''
    Returns the index corresponding to the biggest number in the list, in the form of a two-digit string
    '''
    m, am = l[0], 0
    for i in range(len(l)):
        if l[i] > m:
            m = l[i]
            am = i
    if am < 10:
        return "0"+str(am)
    return str(am)


def modal_string(l):
    '''
    Returns the modal string of the list l, that is, the 3 indices corresponding to the 3 biggest numbers in the list
    '''
    modal = ""
    for i in range(3):
        modal += argmax(l)
        l[int(argmax(l))] = -float("inf")
    return modal

print(modal_string(simulate(50000)))

101524
